In [1]:
from datasets import load_dataset, load_from_disk
import pandas as pd
import ast

Rebel

In [5]:
dataset = load_from_disk("triplets/Rebel_train_0000")
df=dataset.to_pandas()

df["triplets"] = df["triplets"].str.replace("<obj>", "<rel>")
df["triplets"] = df["triplets"].str.replace("<subj>", "<obj>")
df["triplets"] = df["triplets"].str.replace("<triplet>", "<subj>")
df.to_parquet("triplets/preprocessed/Rebel_train_0000.parquet")

In [6]:
df.head(5)

,id,title,context,triplets
0,Q7185360-1,Philippine one hundred-peso note,The Philippine one hundred-peso note ( Filipin...,<subj> Philippine one hundred-peso note <obj> ...
1,Q7185360-2,Philippine one hundred-peso note,Philippine president Manuel A . Roxas is curre...,<subj> Manuel A. Roxas <obj> Philippine presid...
2,Q7185306-1,Philippine five hundred-peso note,The Philippine five hundred-peso note ( Filipi...,<subj> Philippine five hundred-peso note <obj>...
3,Q7185306-3,Philippine five hundred-peso note,"and his wife , President Corazon Aquino are cu...",<subj> Corazon Aquino <obj> President <rel> po...
4,Q7185363-1,Philippine one thousand-peso note,The Philippine one thousand-peso note ( Filipi...,<subj> Philippine one thousand-peso note <obj>...


In [ ]:
import pickle
import ast
from datasets import load_dataset

def get_relations_entities_rebel_from_hf(dataset, rebel_entities=None, rebel_relations=None, entity_file=None, relation_file=None, save=True):
    if rebel_entities is None:
        rebel_entities = set()
    if rebel_relations is None:
        rebel_relations = set()
    
    for obj in dataset:
        for triplet in obj["triplets"]:
            # 把字符串形式的字典转换为真正的字典
            subject = ast.literal_eval(triplet["subject"])
            predicate = ast.literal_eval(triplet["predicate"])
            object_ = ast.literal_eval(triplet["object"])
            
            subject_id = subject.get("uri", "")
            predicate_id = predicate.get("uri", "")
            object_id = object_.get("uri", "")
            
            if subject_id.startswith("Q"):
                rebel_entities.add(subject_id)
            if object_id.startswith("Q"):
                rebel_entities.add(object_id)
            if predicate_id.startswith("P"):
                rebel_relations.add(predicate_id)
    
    if save:
        if entity_file:
            with open(entity_file, "wb") as f:
                pickle.dump(rebel_entities, f)
        if relation_file:
            with open(relation_file, "wb") as f:
                pickle.dump(rebel_relations, f)
    
    return rebel_entities, rebel_relations

dataset = load_dataset("martinjosifoski/SynthIE", "rebel", split="train")
entities, relations = get_relations_entities_rebel_from_hf(
    dataset, 
    entity_file="rebel_entities.pkl", 
    relation_file="rebel_relations.pkl"
)

print(f"get {len(entities)} entities and {len(relations)} relations")


KeyboardInterrupt: 

SynthIE

In [7]:

def convert_triplets(triplets_data):
    triplets_data = ast.literal_eval(triplets_data) if isinstance(triplets_data, str) else triplets_data
    formatted_triplets = []
    
    for item in triplets_data:
        subj_dict = ast.literal_eval(item['subject'])
        pred_dict = ast.literal_eval(item['predicate'])
        obj_dict = ast.literal_eval(item['object'])
        triplet_str = f"<sub> {subj_dict['surfaceform']} <rel> {pred_dict['surfaceform']} <obj> {obj_dict['surfaceform']}"
        formatted_triplets.append(triplet_str)
    
    return formatted_triplets


In [8]:
dataset = load_from_disk("triplets/SynthIE_text_test")
df=dataset.to_pandas()
df['triplets'] = df['triplets'].apply(convert_triplets)
df.to_parquet("triplets/preprocessed/synthie_test.parquet", index=False)

In [9]:
df.head(5)

,id,text,triplets,entities,relations
0,18997,Pickin' Up the Pieces (Fitz and the Tantrums a...,[<sub> Pickin'_Up_the_Pieces_(Fitz_and_the_Tan...,[{'surfaceform': 'Pickin'_Up_the_Pieces_(Fitz_...,"[{'surfaceform': 'record label', 'uri': 'P264'..."
1,44171,Primarily Jazz was recorded at Fullerton College.,[<sub> Primarily_Jazz <rel> recording location...,"[{'surfaceform': 'Primarily_Jazz', 'uri': 'Q72...","[{'surfaceform': 'recording location', 'uri': ..."
2,49548,Njombe Region is an instance of a region in Ta...,[<sub> Njombe_Region <rel> capital <obj> Njomb...,"[{'surfaceform': 'Njombe_Region', 'uri': 'Q208...","[{'surfaceform': 'capital', 'uri': 'P36'}, {'s..."
3,49269,"Konstadinos Anastasiou is a Greek athlete, spe...",[<sub> Konstadinos_Anastasiou <rel> sport <obj...,"[{'surfaceform': 'Konstadinos_Anastasiou', 'ur...","[{'surfaceform': 'sport', 'uri': 'P641'}, {'su..."
4,856,Harold Schwartz is a human who is a citizen of...,[<sub> Harold_Schwartz <rel> country of citize...,"[{'surfaceform': 'Harold_Schwartz', 'uri': 'Q6...","[{'surfaceform': 'country of citizenship', 'ur..."
